## TF-IDF

In [1]:
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [10]:
import sys         
sys.path.append('/home/pranisaa/working_dir/Text-Summarization')
from config import *
import pandas as pd
from datasets import load_dataset
from dataset import Dataset
from transformers import T5Tokenizer, BartTokenizer
pd.set_option('display.max_colwidth', None)

#### TEST TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
text_tfidf = ['my name is beau', 'my name is fabby', 'i love deep learning']
tfidf_vectorizer = TfidfVectorizer(stop_words='english', use_idf=True) #use_idf bool, default=True (to highlight by comparison) Enable inverse-document-frequency reweighting
x = tfidf_vectorizer.fit_transform(text_tfidf)
print(x.toarray().shape)
# print(tfidf_vectorizer.get_feature_names().shape)
#         tfidfcounts = pd.DataFrame(x.toarray(),index = tfidf_vectorizer.get_feature_names(), columns = ["tfidf"])
tfidfcounts = pd.DataFrame(x.toarray(),index = text_tfidf,  columns = tfidf_vectorizer.get_feature_names())
tfidfcounts

(3, 5)


/home/pranisaa/thesis/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,beau,deep,fabby,learning,love
my name is beau,1.0,0.00000,0.0,0.00000,0.00000
my name is fabby,0.0,0.00000,1.0,0.00000,0.00000
i love deep learning,0.0,0.57735,0.0,0.57735,0.57735


### Load dataset

In [7]:
# Load dataset
data = 'xsum'

if data == 'cnn_dailymail':
    dataset = load_dataset(data, '3.0.0')
    source_text = "article"
    target_text = "highlights"
elif data == "xsum":
    dataset = load_dataset(data)
    source_text = "document"
    target_text = "summary"
else:
    raise ValueError("Undefined dataset")

Using the latest cached version of the module from /home/pranisaa/.cache/huggingface/modules/datasets_modules/datasets/xsum/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934 (last modified on Wed Jan 19 12:12:25 2022) since it couldn't be found locally at xsum., or remotely on the Hugging Face Hub.
Using custom data configuration default
Reusing dataset xsum (/home/pranisaa/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

### Get dataset

In [82]:
# import pickle
# with open('max_512_q1.pkl', 'rb') as handle:
#     b = pickle.load(handle)
    
pickle_in = open("val_len_q1_max512.pickle","rb")
example_dict = pickle.load(pickle_in)
q1_id = example_dict['ids']

pickle_in = open("val_len_q3_max512.pickle","rb")
example_dict = pickle.load(pickle_in)
q3_id = example_dict['ids']

In [83]:
# # path = f"""../model/{model_params["MODEL"]}_{data}_lenrestriction_30epoch"""
# # path_df = os.path.join(path, f"""result_eval/predictions_{model_params["MODEL"]}_epoch{model_params["TRAIN_EPOCHS"]-1}.csv""")
# path_df = 'max_512_q1.csv'
# df = pd.read_csv(path_df, reset_index = False)
# df_dict = df.to_dict()
# # q1_id = df_dict['q1']['ids']
# # q3_id = df_dict['q3']['ids']

In [ ]:
df_data = {'id': dataset['validation']['id'], 'document': dataset['validation']['document']}
df_data = pd.DataFrame.from_dict(df_data)

In [53]:
print(len(doc_id))

1000


In [89]:
doc_val_q1 = []
doc_idex_q1 = []
for i in range(len(df_data['id'])):
    if int(df_data['id'][i]) in list(q1_id):
        doc_val_q1.append(df_data['document'][i])
        doc_idex_q1.append(df_data['id'][i])

In [96]:
doc_val_q3 = []
doc_idex_q3 = []
for i in range(len(df_data['id'])):
    if int(df_data['id'][i]) in list(q3_id):
#         print(df_data['id'][i])
        doc_val_q3.append(df_data['document'][i])
        doc_idex_q3.append(df_data['id'][i])

In [91]:
df_data['id'][0]

'38295789'

In [92]:
print(len(doc_val_q1))
print(len(doc_idex_q3))

100
102


In [103]:
from gensim import corpora, models
from sklearn.decomposition import LatentDirichletAllocation
# from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [107]:
tf_vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)
gps_news_matrix = tf_vectorizer.fit_transform(doc_val_q1)
lda = LatentDirichletAllocation(n_components=5, learning_method='online', random_state=0, verbose=0, n_jobs = -1)
lda_model = lda.fit(gps_news_matrix)
lda_matrix = lda_model.transform(gps_news_matrix)
lda_matrix

array([[0.02041554, 0.91834519, 0.02039711, 0.02039324, 0.02044891],
       [0.03109351, 0.03136772, 0.87511378, 0.03116803, 0.03125696],
       [0.03295945, 0.03296886, 0.86810711, 0.03298163, 0.03298296],
       [0.0239644 , 0.02404003, 0.02396178, 0.90401879, 0.024015  ],
       [0.02162763, 0.02167016, 0.02159751, 0.02165304, 0.91345166],
       [0.02993633, 0.02995392, 0.02994845, 0.03005026, 0.88011104],
       [0.02557655, 0.0255959 , 0.02558839, 0.02557978, 0.89765938],
       [0.02461403, 0.02468917, 0.024617  , 0.90137548, 0.02470432],
       [0.86091347, 0.03480663, 0.03472497, 0.03472278, 0.03483215],
       [0.02187807, 0.02189361, 0.02188416, 0.91239307, 0.02195109],
       [0.01902714, 0.92376041, 0.01902471, 0.01908914, 0.0190986 ],
       [0.03218625, 0.0321807 , 0.03215722, 0.03217095, 0.87130488],
       [0.03191388, 0.87207729, 0.03208414, 0.03192203, 0.03200266],
       [0.02913857, 0.88343734, 0.02912897, 0.02914299, 0.02915212],
       [0.01836757, 0.01843153, 0.

In [105]:
def print_topics(model, count_vectorizer, n_top_words):
    words = tf_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
      
        print("\nTopic #%d:" % topic_idx )
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda_model, gps_news_matrix, 15)

Topics found via LDA:

Topic #0:
defender tree harzi scotland said incident nuclear enclosure larne bank cycling barcelona british branches korea

Topic #1:
said police year court ireland scottish mr world paul rescue king time season special league

Topic #2:
said river bbc people hoard club bridge coins mr apple armagh bristol fish local city

Topic #3:
said police government mr park year people turkey asylum information mining smith years security inquiry

Topic #4:
said police called man box baby year pub road games family time pronounced death property


In [108]:
print_topics(lda_model, gps_news_matrix, 15)


Topic #0:
mr year ms duffy said hilary death brunner schwartzel people yellen sharif police woman hiv

Topic #1:
said year coin water play going staff rmt mr bailey flanagan stations season tuesday told

Topic #2:
said car bruno people saw yorkshire movie like hrw head space scenery wildlife terry half

Topic #3:
shares year said rose oil shortman minutes scotland students clyde film 11 dunbartonshire seminary cyber

Topic #4:
mr said device points playback city supported media james amateur just work place irving pemberton


/home/pranisaa/thesis/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [106]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, perplexity=50, learning_rate=100, 
                        n_iter=1000, verbose=1, random_state=0, angle=0.75)
tsne_features = model.fit_transform(lda_matrix)
df = pd.DataFrame(tsne_features)
df['topic'] = lda_matrix.argmax(axis=1)
df.columns = ['TSNE1', 'TSNE2', 'topic']
import seaborn as sns
plt.figure(figsize=(15, 10))
plt.title('T-SNE plot of different headlines ( headlines are clustered among their topics)')
ax = sns.scatterplot(x = 'TSNE1', y = 'TSNE2', hue = 'topic', data = df, legend = 'full')
plt.show()

/home/pranisaa/thesis/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(


[t-SNE] Computing 101 nearest neighbors...
[t-SNE] Indexed 102 samples in 0.000s...
[t-SNE] Computed neighbors for 102 samples in 0.005s...
[t-SNE] Computed conditional probabilities for sample 102 / 102
[t-SNE] Mean sigma: 0.783573
[t-SNE] KL divergence after 250 iterations with early exaggeration: 47.897194
[t-SNE] KL divergence after 1000 iterations: 0.020470


ModuleNotFoundError: No module named 'seaborn'

In [39]:
model_params = {
    "MODEL": "bart-base",  # model_type: t5-base/t5-large
    "BATCH_SIZE": 1,  # training batch size
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 36,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

train_dataset = dataset["train"]
print("Total training set len: ", len(train_dataset))

# Define portion due to RAM limitation
portion = 6
tfidf_data_size = len(train_dataset)//portion

tokenizer = BartTokenizer.from_pretrained(f'facebook/{model_params["MODEL"]}') # just because our Dataset requires it
training_set = Dataset(
    train_dataset[: tfidf_data_size],
    tokenizer,
    model_params["MODEL"],
    model_params["MAX_SOURCE_TEXT_LENGTH"],
    model_params["MAX_TARGET_TEXT_LENGTH"],
    source_text,
    target_text,
)
print("Final data set len: ", len(training_set))

Total training set len:  204045
Final data set len:  34007


In [40]:
df_tfidf = training_set.compute_tfidf()
df_tfidf_mean = df_tfidf.mean(axis=0).nlargest(500)

/home/pranisaa/thesis/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [42]:
df_tfidf_mean

said           0.037405
mr             0.019586
year           0.016670
people         0.015084
police         0.015080
                 ...   
development    0.002504
growth         0.002499
performance    0.002498
michael        0.002498
issues         0.002492
Length: 500, dtype: float64

In [49]:
to_mask_list = []
for i,v in enumerate(df_tfidf_mean):
#     print(df_tfidf_mean.index[i])
#     print(v)
    to_mask_list.append(df_tfidf_mean.index[i])
#     print(df_tfidf_mean.index[i])

In [50]:
# to_mask_list

In [45]:
# import required modules
import spacy
nlp = spacy.load("en_core_web_sm")


nouns = []
verbs = []
adjs = []
advs = []
cardinals = []
others = []
for word in to_mask_list:
  
    # returns a document of object
    tag = nlp(word)
    if "NN" in tag[0].tag_ :
        nouns.append(word)
    elif "VB" in tag[0].tag_ :
        verbs.append(word)
    elif "JJ" in tag[0].tag_ :
        adjs.append(word)
    elif "RB" in tag[0].tag_ :
        advs.append(word)
    elif "CD" in tag[0].tag_ :
        cardinals.append(word)
    else:
        others.append((word,tag[0].tag_))
#     # checking if it is a noun or not
#     if(tag[0].tag_ == 'NNP'):
#         print(text, " is a noun.")
# else:
#     print(text, " is not a noun.")

In [46]:
print("len of nouns: ", len(nouns))
print("len of verbs: ", len(verbs))
print("len of adjs: ", len(adjs))
print("len of others: ", len(others))

len of nouns:  276
len of verbs:  117
len of adjs:  52
len of others:  6


# Spacy Tag
'''
    ".": "punctuation mark, sentence closer",
    ",": "punctuation mark, comma",
    "-LRB-": "left round bracket",
    "-RRB-": "right round bracket",
    "``": "opening quotation mark",
    '""': "closing quotation mark",
    "''": "closing quotation mark",
    ":": "punctuation mark, colon or ellipsis",
    "$": "symbol, currency",
    "#": "symbol, number sign",
    "AFX": "affix",
    "CC": "conjunction, coordinating",
    "CD": "cardinal number",
    "DT": "determiner",
    "EX": "existential there",
    "FW": "foreign word",
    "HYPH": "punctuation mark, hyphen",
    "IN": "conjunction, subordinating or preposition",
    "JJ": "adjective (English), other noun-modifier (Chinese)",
    "JJR": "adjective, comparative",
    "JJS": "adjective, superlative",
    "LS": "list item marker",
    "MD": "verb, modal auxiliary",
    "NIL": "missing tag",
    "NN": "noun, singular or mass",
    "NNP": "noun, proper singular",
    "NNPS": "noun, proper plural",
    "NNS": "noun, plural",
    "PDT": "predeterminer",
    "POS": "possessive ending",
    "PRP": "pronoun, personal",
    "PRP$": "pronoun, possessive",
    "RB": "adverb",
    "RBR": "adverb, comparative",
    "RBS": "adverb, superlative",
    "RP": "adverb, particle",
    "TO": 'infinitival "to"',
    "UH": "interjection",
    "VB": "verb, base form",
    "VBD": "verb, past tense",
    "VBG": "verb, gerund or present participle",
    "VBN": "verb, past participle",
    "VBP": "verb, non-3rd person singular present",
    "VBZ": "verb, 3rd person singular present",
    "WDT": "wh-determiner",
    "WP": "wh-pronoun, personal",
    "WP$": "wh-pronoun, possessive",
    "WRB": "wh-adverb",
    "SP": "space (English), sentence-final particle (Chinese)",
    "ADD": "email",
    "NFP": "superfluous punctuation",
    "GW": "additional word in multi-word expression",
    "XX": "unknown",
    "BES": 'auxiliary "be"',
    "HVS": 'forms of "have"',
    "_SP": "whitespace",
'''

#### Save

In [47]:
import pickle

with open("to_mask_list_top500", "wb") as fp:   #Pickling
    pickle.dump(to_mask_list, fp)

### Result Evaluation

T5-small xsum
- get samples with low rouge scores
- get samples with high rouge scores
- tf-idf